In [1]:
from lxml import html
import lxml
import time
import requests
import pandas as pd
import sqlite3

In [2]:
date_jour = '2017-12-31'
OUTPUT_PATH = '../_outputs/'
database_file = OUTPUT_PATH + 'chefs_{}.db'.format(date_jour)
conn = sqlite3.connect(database_file)

In [3]:
df_source = pd.read_sql_query("select * from index_chefs;", conn)
df_source.head()

,index,chef,total
0,0,florent-gouverneur,14
1,1,arnaud-bulliot,16
2,2,quentin-girerd,2
3,3,dario-jarel,12
4,4,la-belle-assiette-france,540


In [4]:
def getChefPage(chef_id):
    url = 'https://labelleassiette.fr/' + chef_id
    print("Page : " + url)
    response = requests.get(url)
    content_source = response.text
    return [chef_id, html.document_fromstring(content_source)]

In [5]:
docs = []
for index, row in df_source.iterrows():
    docs.append(getChefPage(row['chef']))

Page : https://labelleassiette.fr/florent-gouverneur
Page : https://labelleassiette.fr/arnaud-bulliot
Page : https://labelleassiette.fr/quentin-girerd
Page : https://labelleassiette.fr/dario-jarel
Page : https://labelleassiette.fr/la-belle-assiette-france
Page : https://labelleassiette.fr/kevin-bense
Page : https://labelleassiette.fr/sacharanin-son-herlidou
Page : https://labelleassiette.fr/daniel-lopez
Page : https://labelleassiette.fr/raphael-courtin
Page : https://labelleassiette.fr/ella-aflalo
Page : https://labelleassiette.fr/mariana-delubac
Page : https://labelleassiette.fr/mickael-houx
Page : https://labelleassiette.fr/pierre-guichon
Page : https://labelleassiette.fr/julie-anne-charton
Page : https://labelleassiette.fr/cedric-fortes
Page : https://labelleassiette.fr/antoine-collin
Page : https://labelleassiette.fr/mojgan-tashvighi
Page : https://labelleassiette.fr/luc-fischer
Page : https://labelleassiette.fr/thomas-major
Page : https://labelleassiette.fr/hina-malik
Page : https

Page : https://labelleassiette.fr/stephane-oulevey
Page : https://labelleassiette.fr/eleonor-ill-walgener
Page : https://labelleassiette.fr/elise-deat
Page : https://labelleassiette.fr/herve-escobar
Page : https://labelleassiette.fr/sombat-et-gregor
Page : https://labelleassiette.fr/zelikha-dinga
Page : https://labelleassiette.fr/eleonora-galasso
Page : https://labelleassiette.fr/tomoko-ogata
Page : https://labelleassiette.fr/hanyen-hsu
Page : https://labelleassiette.fr/yuan-ren
Page : https://labelleassiette.fr/noemi-ito
Page : https://labelleassiette.fr/ji-hye-park
Page : https://labelleassiette.fr/mattia-salvadore
Page : https://labelleassiette.fr/jonathan-lefevre
Page : https://labelleassiette.fr/aurelie-scarabin
Page : https://labelleassiette.fr/fanny-matuszkiewicz
Page : https://labelleassiette.fr/aurelie-gauvin
Page : https://labelleassiette.fr/laura-celine-you
Page : https://labelleassiette.fr/boris-banicles
Page : https://labelleassiette.fr/arthur-lefranc
Page : https://labell

In [17]:
len(docs)

284

In [18]:
docs[0][1]

<Element html at 0x11930da48>

In [78]:
def get_rows(doc):
    ratings = doc.xpath('//div[@class="feedback"]//span[@itemprop="ratingValue"]/@content')
    authors = doc.xpath('//div[@class="feedback"]//span[@itemprop="author"]/@content')
    authors_types = doc.xpath('//div[@class="feedback"]//span[2]/text()')
    dates = doc.xpath('//div[@class="feedback"]//span[@itemprop="datePublished"]/@content')
    descriptions = doc.xpath('//div[@class="feedback"]//p[@itemprop="description"]')
    
    return ratings, authors, authors_types, dates, descriptions
    # return len(ratings), len(authors), len(authors_types), len(dates), len(descriptions)

In [79]:
# get_rows(docs[9][1])

In [80]:
docs[36][0]

'noel-tellier'

In [81]:
columns = ['chef', 'rating', 'author', 'author_type', 'dates', 'description']
df_output = pd.DataFrame(columns=columns)

In [90]:
idx = 0
for doc in docs:
    ratings, authors, authors_types, dates, descriptions = get_rows(doc[1])
    chef = doc[0]
    # print(chef)
    idxm = 0
    for r in ratings:
        descriptions[idxm] = descriptions[idxm].text_content()
        df_output.loc[idx] = [chef, r, authors[idxm], authors_types[idxm], dates[idxm], descriptions[idxm]]
        idx +=1
        idxm +=1

In [91]:
df_output.head(50)

,chef,rating,author,author_type,dates,description
0,florent-gouverneur,5,Fabio C.,(Hôte),2017-12-19,"Tout parfait.\rPonctuel, repas parfait, très d..."
1,florent-gouverneur,4,Clémence B.,(Hôte),2017-12-17,Super expérience
2,florent-gouverneur,5,Fanny O.,(Hôte),2017-11-23,"Une très belle expérience, un délicieux repas,..."
3,florent-gouverneur,4,Catherine G.,(Hôte),2017-10-29,"Discrétion, professionnalisme et efficacité du..."
4,florent-gouverneur,5,Pierre-alexandre B.,(Hôte),2017-10-19,Très belle prestation !
5,florent-gouverneur,4,Audrey M.,(Hôte),2017-10-17,"Ce fut un moment très sympathique, beaucoup de..."
6,florent-gouverneur,4,Nathalie D.,(Hôte),2017-10-12,Excellente
7,florent-gouverneur,4,Nesrine B.,(Invité),2017-07-23,Nous avons passé une excellente soirée avec me...
8,florent-gouverneur,4,Mehdi B.,(Invité),2017-07-23,Le repas était très bon de l'amuse bouche au d...
9,arnaud-bulliot,5,Aymeric G.,(Hôte),2017-12-23,Très bonne prestation


In [92]:
df_output['description'][30]

"rien à dire c'était parfait "

In [93]:
df_output.to_excel(OUTPUT_PATH + 'chefs_{}_avis.xlsx'.format(date_jour), index=False, header=True)